In [ ]:
import re

import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt

from health_tracking import AppleHealthParser

sns.set()
%matplotlib inline

In [ ]:
parser = AppleHealthParser()

In [ ]:
workouts_df, workout_types = parser.extract_workouts()
workout_types

Pick the recorded running workouts.

In [ ]:
running_df = workouts_df[workouts_df["workoutActivityType"] == "Running"].copy()
running_df.head()

#### Create `minutesPerKm` Column

In [ ]:
def minutes_per_km(row):
    result = 0
    try:
        result = row.duration / row.totalDistance
    
    except:
        pass
    
    return result

running_df["minutesPerKm"] = running_df.apply(minutes_per_km, axis=1)
running_df.head()

#### Remove Outliers

Removing values above 8 and below 4 minutes/km.

In [ ]:
running_df = running_df[(running_df["minutesPerKm"] <= 8) & (running_df["minutesPerKm"] >= 4)]

#### Plot `minutesPerKm`

In [ ]:
sns.jointplot(range(running_df.shape[0]), "minutesPerKm", data=running_df, kind="reg", xlim=(-5, running_df.shape[0] + 5))

#### Use Natural Distance for Days

In [ ]:
day_zero = running_df.creationDate[0]
day_offset = running_df.creationDate[running_df.shape[0]] - running_df.creationDate[0]

In [ ]:
running_df["dayOffset"] = running_df.apply(lambda row: (row.creationDate - day_zero).days, axis=1)

In [ ]:
jointplot = sns.jointplot("dayOffset", "minutesPerKm", data=running_df, kind="reg", xlim=(-10, running_df.dayOffset.max() + 10))